In [1]:
import torch

print(torch.backends.mps.is_available())  # Should print True
print(torch.backends.mps.is_built())      # Should print True

import torch.nn as nn
from torch.nn import functional as F

import requests

device = torch.device("mps")
print(device)

True
True
mps


In [2]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 1000
eval_interval = 50
learning_rate = 1e-2
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

In [3]:
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
r = requests.get(url=url)
text = r.text

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
  # generate a small batch of data of inputs x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

In [4]:
class Head(nn.Module):
  """ one head of self-attention """

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)   # (B,T,C)
    q = self.query(x) # (B,T,C)
    # compute attention scores ("affinities")
    wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    wei = self.dropout(wei)
    # perform the weighted aggregation of the values
    v = self.value(x) # (B,T,C)
    out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
    return out

class MultiHeadAttention(nn.Module):
  """ multiple heads of self-attention in parallel """

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out

class FeedFoward(nn.Module):
  """ a simple linear layer followed by a non-linearity """

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(n_embd, 4 * n_embd),
      nn.ReLU(),
      nn.Linear(4 * n_embd, n_embd),
      nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):
  """ Transformer block: communication followed by computation """

  def __init__(self, n_embd, n_head):
    # n_embd: embedding dimension, n_head: the number of heads we'd like
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedFoward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

# super simple bigram model
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd) # final layer norm
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    # idx and targets are both (B,T) tensor of integers
    tok_emb = self.token_embedding_table(idx) # (B,T,C)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
    x = tok_emb + pos_emb # (B,T,C)
    x = self.blocks(x) # (B,T,C)
    x = self.ln_f(x) # (B,T,C)
    logits = self.lm_head(x) # (B,T,vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # crop idx to the last block_size tokens
      idx_cond = idx[:, -block_size:]
      # get the predictions
      logits, loss = self(idx_cond)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # print the generated character
      print(decode([idx_next[0].item()]), end='', flush=True)
      # append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    print() # new line at the end

model = BigramLanguageModel()
m = model.to(device)

In [5]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [6]:
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
  if iter % eval_interval == 0 or iter == max_iters - 1:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 50: train loss 2.5932, val loss 2.5915
step 100: train loss 2.5191, val loss 2.5183
step 150: train loss 2.3992, val loss 2.4194
step 200: train loss 2.3402, val loss 2.3436
step 250: train loss 2.2770, val loss 2.2938
step 300: train loss 2.2212, val loss 2.2536
step 350: train loss 2.1889, val loss 2.2120
step 400: train loss 2.1619, val loss 2.1912
step 450: train loss 2.1383, val loss 2.1823
step 500: train loss 2.1124, val loss 2.1641
step 550: train loss 2.0845, val loss 2.1388
step 600: train loss 2.0510, val loss 2.1222
step 650: train loss 2.0510, val loss 2.1225
step 700: train loss 2.0250, val loss 2.1212
step 750: train loss 2.0023, val loss 2.0983
step 800: train loss 1.9851, val loss 2.0615
step 850: train loss 1.9749, val loss 2.0859
step 900: train loss 1.9645, val loss 2.0610
step 950: train loss 1.9530, val loss 2.0445
step 999: train loss 1.9292, val loss 2.0488


In [7]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
model.generate(context, max_new_tokens=1500)

Foastay do fee chaed the be usfriie.

MENRY MERCORIONIUBROKETRORK:
My comer his noblone avrious Lack,
Wy boid pray you ar home but, fas is shalk, and ih Nary
But aw to nef aw it freelbustselown'd slom of to gartly fall me fajerise.

And thelp welt ens? Wither for he gientyou

MEOLTo:
Cerile 'Rwureis latirray noks, he
To do murs, in my you his yought, elp thought;
And have fill ruch and as was im to, and my rett daties', and that reatan and sule his Kink, er her known:
By to Kathis dided the kind aguef:
Wifd cith him necltil my herroy-pried and I and
Tyconle-huther your will ropt it.

LANUMELLO:
Aid my bew!
Asteet farly ras areps wouldo mose fath may,
Where; my gll spry arm thim, hent, arthe and broth of he pitighteer.

LEONA:
By son.

HENMAMING RICHARD IUS:
Nhat, alk, foutull councaep up is is man
So MIIANUS:

LANE:
But, me cher, prom, her!

LEONA:
Wather tom, ther, is by thy was own was tall sirely contlought
As hath en him to-sired is theer aster
Has fought:
WholO to so me og I and h

In [ ]:
torch.save(model.state_dict(), 'nietzsche_jgpt_weights.pth')

saved_model = BigramLanguageModel().to(device)
saved_model.load_state_dict(torch.load('nietzsche_jgpt_weights.pth'))
saved_model.eval()

In [ ]:


# test_context = torch.zeros((1, 1), dtype=torch.long, device=device)

# print(
#   decode(
#     saved_model.generate(test_context, max_new_tokens=2000)[0].tolist()
#   )
# )